In [3]:
import sklearn
import pandas as pd
import numpy as np

In [56]:
df = pd.read_csv('../data/features.csv')

In [57]:
df = df.set_index(keys='id')

In [62]:
df

,sharpness,saturation,brightness,entropy,contrast,colorfulness
id,,,,,,
63,45.042899,60.625145,3.839637,965.051933,76.142303,62.953686
1353,65.857150,72.958046,4.571306,211.078292,58.294239,97.960309
1421,44.831194,58.877440,4.714785,745.972724,66.019640,52.555248
2896,42.557714,67.311302,4.013180,117.061072,57.933238,54.319254
610,47.668126,60.862984,4.908555,700.064150,64.062041,60.220316
...,...,...,...,...,...,...
4232,31.309838,73.742096,3.318515,21.122673,45.294859,62.539970
190,56.064134,60.471814,4.153673,155.951339,47.511836,71.039435
1404,34.334820,67.380092,3.769233,439.794763,64.077598,48.508521


#### Checking Simularities:

Euclidian score:
$$d(user_{i}, user_{j}) = \sqrt{(x_{1} - x_{2})^2 + (y_{1} - y_{2})^2}$$

Pearson Correllation
$$r = \frac{\sum(x_{i} - \bar{x})(y_{i} - \bar{y})}{\sqrt{\sum(x_{i} - \bar{x})^2\sum(y_{i} - \bar{y})^2}}$$

In [64]:
#Function to compute Euclidean Distance. 
def euclidean(v1, v2):
    
    #Convert 1-D Python lists to numpy vectors
    v1 = np.array(v1)
    v2 = np.array(v2)
    
    #Compute vector which is the element wise square of the difference
    diff = np.power(np.array(v1)- np.array(v2), 2)
    
    #Perform summation of the elements of the above vector
    sigma_val = np.sum(diff)
    
    #Compute square root and return final Euclidean score
    euclid_score = np.sqrt(sigma_val)
    
    return euclid_score

In [66]:
euclidean(df.iloc[0], df.iloc[1])

755.3847609591617

In [28]:
def cosine_similarity(a,b):
    return np.dot(a,b) / ( (np.dot(a,a) **.5) * (np.dot(b,b) ** .5) )
cosine_similarity(a,b)

0.22850928401946777

In [30]:
df_ranked = pd.DataFrame(columns=['id', 'simularity'])

## The recommender

Step one is getting our item ratings

In [85]:
df = pd.read_csv('../data/features.csv')

In [86]:
df.head()

,sharpness,saturation,brightness,entropy,contrast,colorfulness,id
0,45.042899,60.625145,3.839637,965.051933,76.142303,62.953686,63
1,65.857150,72.958046,4.571306,211.078292,58.294239,97.960309,1353
2,44.831194,58.877440,4.714785,745.972724,66.019640,52.555248,1421
3,42.557714,67.311302,4.013180,117.061072,57.933238,54.319254,2896
4,47.668126,60.862984,4.908555,700.064150,64.062041,60.220316,610


Set misc commands:

In [87]:
df = df.set_index(keys='id')
df.values

array([[  45.04289882,   60.62514475,    3.83963689,  965.05193304,
          76.14230303,   62.9536859 ],
       [  65.85715033,   72.95804575,    4.57130625,  211.07829212,
          58.29423947,   97.96030895],
       [  44.83119407,   58.87744   ,    4.71478489,  745.97272405,
          66.01963993,   52.55524774],
       ...,
       [  34.33482039,   67.38009237,    3.76923333,  439.79476275,
          64.07759846,   48.50852148],
       [  33.43680163,   72.1566353 ,    4.83479604, 3468.32106294,
          69.00547578,   62.40768953],
       [  65.49526825,   52.00348924,    3.82627903,   93.95414966,
          59.85152931,   56.08370859]])

In [94]:
df.iloc[1].values
df.iloc[1].index

Index(['sharpness', 'saturation', 'brightness', 'entropy', 'contrast',
       'colorfulness'],
      dtype='object')

In [71]:
def get_item_profile(item_id):
    return df.loc[df['id']==item_id]

In [73]:
get_item_profile(63)

,sharpness,saturation,brightness,entropy,contrast,colorfulness,id
0,45.042899,60.625145,3.839637,965.051933,76.142303,62.953686,63
